In [1]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [2]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `input/parameters.xls` file. This file also includes scenarios as columns.

In [3]:
kwargs = {'workers':4, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [4]:
scenarios = ['base'] #['2035_base', 'CO2-tax_210', 'road_charge']

In [5]:
import time
start = time.time()

# Network preparation

NOT required for the fist model run.

Execute all `prep1` and `prep2` steps. Only neccessary if new network data or assumptions are applied.

In [7]:
# Network generation
parallel_call_notebook('prep10_zones_disagg.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebooks(
    ('prep11_road.ipynb', scenarios),
    ('prep12_rail_coach.ipynb', scenarios),
    ('prep14_bus.ipynb', scenarios),
    **kwargs)
parallel_call_notebook('prep13_air.ipynb', arg_list=scenarios, **kwargs)

0 prep12_rail_coach.py base
1 prep14_bus.py base
subprocess **prep14_bus.py1 base** terminated with an error.
1763 seconds
0 prep13_air.py base
28 seconds


In [8]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)

0 prep15_census_distances.py base
7805 seconds


In [9]:
# Network aggregation and inter-connection
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
# Run the aggregation twice in order to reduce bus trips properly
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

0 prep20_cluster.py base
3099 seconds
0 prep21_aggregate_pt.py base
1220 seconds
0 prep22_access_egress_road.py base
98 seconds


# Fewer zones for modelling on a laptop

If only 8GB RAM is available, the zoning system must be reduced to 401 NUTS3-level zones. The following steps create the corresponding network model.

In [ ]:
parallel_call_notebook('prep10_zones.ipynb', arg_list=scenarios, **kwargs)
# Networks are already computed
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)
# Network aggregation and inter-connection
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

## Sparsify the OD set for a faster, smaller model

Computation of all OD pairs is not required for accurate aggregate results (pkm, choice probabilities, etc.). Hence, the OD set can be sparsified before the pathfinders are executed. The sample size is defined in the general attributes of the `parameters.xls` file. Leave the field empty for no sampling.

If you sample the OD set, you don't need to compute volumes in the 4-step cells below.

In [12]:
# Calculation of the OD matrix
#parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('model_distribution.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('model_volumes_exo.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)

0 model_volumes_endo.py base
572 seconds


# Pathfinders and level of service

Required for the first model run.

If assumptions in `parameters.xls` are changed, the corresponding `prep3X` notebooks should be ran.

Note: All upcoming steps should be run on the same computer, as intermediate model files are pickled and Python's pickle protocols are not stable across different versions.

In [13]:
# Road
parallel_call_notebook('prep30_pathfinder_road.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)

0 prep30_pathfinder_road.py base
327 seconds
0 prep31_prices_car.py base
34 seconds


In [14]:
# PT
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)
# terminates with error from 'assert manual' if there is a saved versions of footpaths
# so that it doesn't have to compute them again

0 prep30_pathfinder_pt.py base
7520 seconds
0 prep31_prices_pt.py base
120 seconds
0 prep33_non-motor.py base
18 seconds


# Estimation

Calculation of parameters for the utility function. Data set access required (see data availability in Readme file).

In [6]:
# Calibration data preparation
# NOTE: only applicapble if you have access to the calibration dataset
#parallel_call_notebook('cal10_input_data.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal1x_validation_data.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('cal11_los_columns_inter-zonal.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal19_vot.ipynb', arg_list=scenarios, **kwargs)

0 cal10_input_data.py base
subprocess **cal10_input_data.py0 base** terminated with an error.
22 seconds
0 cal1x_validation_data.py base
subprocess **cal1x_validation_data.py0 base** terminated with an error.
4 seconds
0 cal11_los_columns.py base
72 seconds


In [7]:
# Estimation mode choice parameters
# Uses all CPU cores
parallel_call_notebook('cal20_estimation_mode.ipynb', arg_list=scenarios, **kwargs)

0 cal20_estimation.py base
5475 seconds


In [ ]:
# Estimation destination choice parameters
# Uses all CPU cores
parallel_call_notebook('cal21_estimation_destination.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Estimation inner/inter zonal choice parameters
# Uses all CPU cores
parallel_call_notebook('cal22_estimation_inner-inter.ipynb', arg_list=scenarios, **kwargs)

# 4-step modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [10]:
# Generation
# Doesn't need to be run for the base scenario, as generation volumes
# are already saved in the zones table
#parallel_call_notebook('model_generation_exo.ipynb', arg_list=scenarios, **kwargs) # fixed probabilities from MiD2017
parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs) # MNL model based on MiD2017

0 model_generation_endo.py base
6009 seconds


In [8]:
# Distribution choice probabilities
parallel_call_notebook('model_destination.ipynb', arg_list=scenarios, **kwargs)

0 model_destination.py base
923 seconds


In [11]:
# Generation of the OD matrix
#parallel_call_notebook('model_volumes_exo.ipynb', arg_list=scenarios, **kwargs) # volumes from VP2030
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)

0 model_volumes_endo.py base
537 seconds


In [15]:
# Mode choice probabilities
parallel_call_notebook('model_logit.ipynb', arg_list=scenarios, **kwargs)

0 model_logit.py base
57 seconds


In [20]:
# Assignment
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)

0 model_assignment.py base
386 seconds


# Equilibration

Demand-supply equilibration happens iteratively for road traffic. It can be used to refine results after running the assignment step once. If so, a new model folder `<scenario name>_equilibrium` is created from which the modelling steps take their road LoS table and save the equilibrated results.

In [ ]:
# Run the equilibrium road pathfinder to create the new LoS table
parallel_call_notebook('model_assignment_equilibrium.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)
# Rename the scenarios
scenarios = [str(s)+'_equilibrium' for s in scenarios]

In [ ]:
# Demand model
parallel_call_notebook('model_logit.ipynb', arg_list=scenarios, **kwargs)
# Supply model
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)

# Post-processing

Calculate transport system indicators such as total pkm, energy demand, or GHG emissions.

In [22]:
# Post-processing
#parallel_call_notebook('post_assignment_inner-zonal.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('post_emissions.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('post_energy.ipynb', arg_list=scenarios, **kwargs)

0 post_emissions.py base
1055 seconds


In [ ]:
# Total execution time (minutes)
int((time.time() - start) / 60)